# Flood Mapping Explained

TODO:
- make setup with EODC Dask Gateway work
- Simplify notebook (aggregate functions, improve descriptions, make it smaller/concise)
Maybe sth like this:
   ``` 
    sig0_dc, hpar_dc, plia_dc = preprocess(bbox, datetime, dynamic)
    flood_dc = calculate_flood_dc(sig0_dc, plia_dc, hpar_dc)
    flood_dc["wbsc"] = calc_water_likelihood(flood_dc)  # Water
    flood_dc["hbsc"] = harmonic_expected_backscatter(flood_dc)  # Land
    ```
- Currently access to Dask EODC Gateway is only granted after requesting additional permission via e-mail
- Add already a predefined number of workers, so the user doesnt need to worry about it
- See if EODC Dask Gateway can handle the amount of participants


In [ ]:
#!pip install dask-flood-mapper

## Set connection to EODC Dask Gateway
Autentication is required through an username and password that should be requested to EODC.

In [ ]:
import os
os.environ["AWS_NO_SIGN_REQUEST"] = "YES"


import hvplot.xarray  # noqa
from dask_flood_mapper import flood
from eodc import settings
from eodc.dask import EODCDaskGateway
from rich.console import Console
from rich.prompt import Prompt
import os
import numpy as np
import pystac_client
import xarray as xr
from dask.distributed import Client, wait
from odc import stac as odc_stac

In [3]:
settings.DASK_URL = "http://dask.services.eodc.eu"
settings.DASK_URL_TCP = "tcp://dask.services.eodc.eu:10000/"

In [ ]:
console = Console()
your_username = Prompt.ask(prompt="Enter your Username")
gateway = EODCDaskGateway(username=your_username)

### Cluster Configuration

It is possible to change the default configuration to optimize your specifications at the cell below.

In [5]:
cluster_options = gateway.cluster_options()
cluster_options.image = "ghcr.io/eodcgmbh/cluster_image:2025.4.1"
cluster_options.worker_cores = 8
cluster_options.worker_memory = 16

<div class="alert alert-block alert-info">
<b>Important:</b> Per default no worker is spawned, therefore please use the widget to add/scale Dask workers in order to enable computations on the cluster.
</div>

In [ ]:
cluster = gateway.new_cluster(cluster_options)
cluster.scale(5) # Or adaptative scale: cluster.adapt(minimum=3, maximum=8)
client = cluster.get_client()
cluster

In [ ]:
# command to check compatibility between client and scheduler
versions = client.get_versions(check=True)


from pprint import pprint
pprint(versions)

#### List available clusters

In [ ]:
console.print(gateway.list_clusters())

#### Connect to already running clusters 

Instead of restarting the whole process again, you can directly connect to cluster already running.

In [10]:
#cluster = gateway.connect(gateway.list_clusters()[0].name)
#console.print(cluster)

In [11]:
#cluster.close(shutdown=True)

### Dask dashboard

The following link displays the Dask Dashboard that can be used to monitor the execution of computations.

In [ ]:
cluster.dashboard_link 

In conjunction with setting up the Dask client qw will chunk the arrays along three dimensions according to the following specifications for maximum performance on my setup. 


In [13]:
chunks = {"time": 1, "latitude": 1300, "longitude": 1300}

## Cube Definitions

The following generic specifications are used for presenting the data.


In [14]:
# Coordinate Reference System - World Geodetic System 1984 (WGS84) in this case
crs = "EPSG:4326"
res = 0.00018  # 20 meter in degree

## Northern Germany Flood

Storm Babet hit the Denmark and Northern coast at the 20th of October 2023 [Wikipedia](https://en.wikipedia.org/wiki/Storm_Babet). Here an area around Zingst at the Baltic coast of Northern Germany is selected as the study area.


In [15]:
time_range = "2023-10-11/2023-10-25"
minlon, maxlon = 12.3, 13.1
minlat, maxlat = 54.3, 54.6
bounding_box = [minlon, minlat, maxlon, maxlat]

## EODC STAC Catalog

The `pystac_client` establishes a connection to the EODC STAC Catalog. This results in a catalog object that can be used to discover collections and items hosted at EODC.


In [16]:
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1")

In [ ]:
from pystac_client import Client

# Open the STAC API
#client = Client.open("https://stac.eodc.eu/api/v1")

# Get all collections
collections = eodc_catalog.get_collections()

# Print each collection ID and title (if available)
for collection in collections:
    print(f"ID: {collection.id}")
    if collection.title:
        print(f"Title: {collection.title}")
    print()


## Microwave Backscatter Measurements

The basic premise of microwave-based backscattering can be seen in the sketch below, the characteristics of backscattering over land and water differ considerably. With this knowledge we can detect whenever a pixel with a predominant land like signature changes to a water like signature in the event of flooding.

![](https://www.gsi.ie/images/images/SAR_mapping_land_water.jpg)

*Schematic backscattering over land and water. Image from [Geological Survey Ireland](https://www.gsi.ie/images/images/SAR_mapping_land_water.jpg)*

We discover Sentinel-1 microwave backscatter ($\sigma_0$ [1]) at a 20 meter resolution, like so:


In [ ]:
search = eodc_catalog.search(
    collections="SENTINEL1_SIG0_20M",
    bbox=bounding_box,
    datetime=time_range,
)

items_sig0 = search.item_collection()
items_sig0

The state of the orbit and relative orbit number is also saved, as the water and land likelihoods (which are calculated later on) dependent on the orbital configuration. These variables will be added as additional coordinates to the data cube. For this purpose a small helper function is defined, like so:


In [19]:
def extract_orbit_names(items):
    return np.array(
        [
            items[i].properties["sat:orbit_state"][0].upper()
            + str(items[i].properties["sat:relative_orbit"])
            for i in range(len(items))
        ]
    )

We will also save the scaling factor and nodata values of STAC items to correct the loaded data accordingly. Again a helper function will be used to correctly scale and fill no data values of the cube.


In [20]:
def post_process_eodc_cube(dc: xr.Dataset, items, bands):
    if not isinstance(bands, tuple):
        bands = tuple([bands])
    for i in bands:
        dc[i] = post_process_eodc_cube_(
            dc[i], items, i
        )  # https://github.com/TUW-GEO/dask-flood-mapper.git
    return dc


def post_process_eodc_cube_(dc: xr.Dataset, items, band):
    scale = items[0].assets[band].extra_fields.get("raster:bands")[0]["scale"]
    nodata = items[0].assets[band].extra_fields.get("raster:bands")[0]["nodata"]
    return dc.where(dc != nodata) / scale

The VV polarization of the discover items can be loaded with `odc-stac` and cast in the desired projection and resolution. The data is at this point only lazily loaded, meaning that we only make an instance of the outlines of the datacube with the proper shape and resolution, but without actually loading all the data. This is done by providing the chunks as defined before, which partitions the data in portions which are more easily handled by the setup used for processing the data (in this case my own pc). 


In [21]:
bands = "VV"
sig0_dc = odc_stac.load(
    items_sig0,
    bands=bands,
    crs=crs,
    chunks=chunks,
    resolution=res,
    bbox=bounding_box,
    resampling="bilinear",
    groupby=None,
)

Now we can rescale our variable, fill the no data values with `np.nan` values, and add the orbit names, with the previous defined functions.


In [22]:
sig0_dc = (
    post_process_eodc_cube(sig0_dc, items_sig0, bands)
    .rename_vars({"VV": "sig0"})
    .assign_coords(orbit=("time", extract_orbit_names(items_sig0)))
    .dropna(dim="time", how="all")
    .sortby("time")
)

Then we remove duplicate time dimensions from the data cube and extract the orbit names as we will need those for obtaining the correct harmonic parameters and local incidence angles,as explained in the next section. Also, note, that we call `dask.persist` to materialize the object but retain it as a delayed object in the workers memory.


In [ ]:
__, indices = np.unique(sig0_dc.time, return_index=True)
indices.sort()
orbit_sig0 = sig0_dc.orbit[indices].data
sig0_dc = sig0_dc.groupby("time").mean(skipna=True)
sig0_dc = sig0_dc.assign_coords(orbit=("time", orbit_sig0))
sig0_dc = sig0_dc.persist()
sig0_dc

## Harmonic Parameters

The so-called likelihoods of $P(\sigma^0|flood)$ and $P(\sigma^0|nonflood)$ can be calculated from past backscattering information. To be able to this we load the harmonic parameters we can model the expected variations in land back scattering based on seasonal changes in vegetation. The procedure is similar to the backscattering routine.

We discover items.


In [24]:
search = eodc_catalog.search(collections="SENTINEL1_HPAR", bbox=bounding_box)

items_hpar = search.item_collection()

Load the data as a lazy object.


In [25]:
bands = ("C1", "C2", "C3", "M0", "S1", "S2", "S3", "STD")
hpar_dc = odc_stac.load(
    items_hpar,
    bands=bands,
    crs=crs,
    chunks=chunks,
    resolution=res,
    bbox=bounding_box,
    groupby=None,
)

hpar_dc = post_process_eodc_cube(hpar_dc, items_hpar, bands).rename({"time": "orbit"})
hpar_dc["orbit"] = extract_orbit_names(items_hpar)
hpar_dc = hpar_dc.groupby("orbit").mean(skipna=True)

We expand the variables along the orbits of sigma naught to be able to calculate the correct land reference backscatter signatures.


In [ ]:
hpar_dc = hpar_dc.sel(orbit=orbit_sig0)
hpar_dc = hpar_dc.persist()
hpar_dc

## Local Incidence Angles

Local incidence angles of measured microwave backscattering is as well important for calculating reference backscatter values, but now for water bodies. The procedure is much the same as for the harmonic parameters.


In [27]:
search = eodc_catalog.search(collections="SENTINEL1_MPLIA", bbox=bounding_box)

items_plia = search.item_collection()

Load the lazy object and preprocess.


In [28]:
bands = "MPLIA"
plia_dc = odc_stac.load(
    items_plia,
    bands=bands,
    crs=crs,
    chunks=chunks,
    resolution=res,
    bbox=bounding_box,
    groupby=None,
)

plia_dc = post_process_eodc_cube(plia_dc, items_plia, bands).rename({"time": "orbit"})
plia_dc["orbit"] = extract_orbit_names(items_plia)
plia_dc = plia_dc.groupby("orbit").mean(skipna=True)

We expand the variables along the orbits of sigma naught to be able to calculate the correct land reference backscatter signatures.


In [ ]:
plia_dc = plia_dc.sel(orbit=orbit_sig0)
plia_dc = plia_dc.persist()
plia_dc

## ESA World Cover from Terrascope

For flood mapping we are only interested in microwave backscattering over what used to be land, as such, we need a way to mask water bodies. For this we use the ESA World Cover data from the following STAC catalog.


In [30]:
os.environ["AWS_NO_SIGN_REQUEST"] = "YES"


Similarly, we discover the required items.


In [ ]:
search_GFM = eodc_catalog.search(
    collections="GFM",
    bbox=bounding_box,
    datetime="2023-10-11T05:33:43.000000000", # choose one time stamp for simplification an avoid errors
)

items_GFM = search_GFM.item_collection()
items_GFM



In [32]:
# resoluton and crs are different from the rest of the data cubes, wll that be a problem to merge?

#resolution = items_GFM[0].properties['gsd']
#print(resolution)

#import pyproj
#crs = pyproj.CRS.from_wkt(items_GFM[0].properties["proj:wkt2"])
#print(crs)

In [33]:
GFM_dc= (odc_stac.load(
    items_GFM, 
    bbox=bounding_box,   # Define the bounding box for the area of interest
    crs=crs,   # Set the coordinate reference system
    bands=["reference_water_mask"],   # Specify the bands to load, comment to load all bands
    resolution=res   # Set the resolution of the data
    #dtype='uint8',   # Define the data type
    #chunks={"x": 1000, "y": 1000, "time": -1},  # Set the chunk size for Dask
    )
    .squeeze("time")
    .drop_vars("time")
    .rename_vars({"reference_water_mask": "wcover"}))

In [ ]:
GFM_dc

Load the data lazily.


## Fuse cube

Here we fuse the four data cubes together and filter for the values that have a HAND value of above zero. We can now drop the obit coordinates as well as time slices which contain no land backscattering data.


In [35]:
import os
os.environ["AWS_NO_SIGN_REQUEST"] = "YES"


In [ ]:
import os
os.environ["AWS_NO_SIGN_REQUEST"] = "YES"

import xarray as xr

flood_dc = xr.merge([sig0_dc, plia_dc, hpar_dc, GFM_dc])
flood_dc = flood_dc.where(flood_dc.wcover != 255)


flood_dc

In [37]:
import os
os.environ["AWS_NO_SIGN_REQUEST"] = "YES"

from odc.stac import configure_rio
configure_rio(aws={"aws_unsigned": True})


flood_dc = (
    flood_dc.reset_index("orbit", drop=True))



In [38]:
flood_dc = flood_dc.rename({"orbit": "time"})


In [39]:
import os
os.environ["AWS_NO_SIGN_REQUEST"] = "YES"

from odc.stac import configure_rio
configure_rio(aws={"aws_unsigned": True})




In [40]:
flood_dc = flood_dc.dropna(dim="time", how="all", subset=["sig0"])

## Likelihoods

Now we are ready to calculate the likelihoods of micorwave backscattering given flooding (or non flooding).

### Water

We start with water which is the simplest calculation, where the hard coded values are coefficients of a regression model fitted to global water backscattering values.  


In [41]:
def calc_water_likelihood(dc):
    return dc.MPLIA * -0.394181 + -4.142015

In [42]:
flood_dc["wbsc"] = calc_water_likelihood(flood_dc)

### Land

For land backscattering we construct the harmonic model from the parameters as contained in the fused data cube. By doing so, we obtain a reference land backscattering value to which to compare our actual observed sigma naught values.


In [43]:
def harmonic_expected_backscatter(dc):
    w = np.pi * 2 / 365

    t = dc.time.dt.dayofyear
    wt = w * t

    M0 = dc.M0
    S1 = dc.S1
    S2 = dc.S2
    S3 = dc.S3
    C1 = dc.C1
    C2 = dc.C2
    C3 = dc.C3
    hm_c1 = (M0 + S1 * np.sin(wt)) + (C1 * np.cos(wt))
    hm_c2 = (hm_c1 + S2 * np.sin(2 * wt)) + C2 * np.cos(2 * wt)
    hm_c3 = (hm_c2 + S3 * np.sin(3 * wt)) + C3 * np.cos(3 * wt)
    return hm_c3

In [44]:
flood_dc["hbsc"] = harmonic_expected_backscatter(flood_dc)

## Flood mapping

Having calculated the likelihoods, we can now move on to calculate the probability of (non-)flooding given a pixel's $\sigma^0$. These so-called *posteriors* need one more piece of information, as can be seen in the equation above. We need the probability that a pixel is flooded $P(F)$ or not flooded $P(NF)$. Of course, these are the figures we've been trying to find this whole time. We don't actually have them yet, so what can we do? In Bayesian statistics, we can just start with our best guess. These guesses are called our "priors", because they are the beliefs we hold *prior* to looking at the data. This subjective prior belief is the foundation Bayesian statistics, and we use the likelihoods we just calculated to update our belief in this particular hypothesis. This updated belief is called the "posterior".

Let's say that our best estimate for the chance of flooding versus non-flooding of a pixel is 50-50: a coin flip.  We now can also calculate the probability of backscattering $P(\sigma^0)$, as the weighted average of the water and land likelihoods, ensuring that our posteriors range between 0 to 1.

The following code block shows how we calculate the priors which allow use to predict whether it is likely if a land pixel became flooded.


In [45]:
def bayesian_flood_decision(dc):
    nf_std = 2.754041
    sig0 = dc.sig0
    std = dc.STD
    wbsc = dc.wbsc
    hbsc = dc.hbsc

    f_prob = (1.0 / (std * np.sqrt(2 * np.pi))) * np.exp(
        -0.5 * (((sig0 - wbsc) / nf_std) ** 2)
    )
    nf_prob = (1.0 / (nf_std * np.sqrt(2 * np.pi))) * np.exp(
        -0.5 * (((sig0 - hbsc) / nf_std) ** 2)
    )

    evidence = (nf_prob * 0.5) + (f_prob * 0.5)
    nf_post_prob = (nf_prob * 0.5) / evidence
    f_post_prob = (f_prob * 0.5) / evidence
    decision = xr.where(
        np.isnan(f_post_prob) | np.isnan(nf_post_prob),
        np.nan,
        np.greater(f_post_prob, nf_post_prob),
    )
    return nf_post_prob, f_post_prob, decision

In [46]:
flood_dc[["nf_post_prob", "f_post_prob", "decision"]] = bayesian_flood_decision(
    flood_dc
)

## Postprocessing

We continue by improving our flood map by filtering out observations that we expect to have low sensitivity to flooding based on a predefined set of criteria.

These criteria include:
* Masking of Exceeding Incidence Angles
* Identification of Conflicting Distributions
* Removal of Measurement Outliers
* Denial of High Uncertainty on Decision


In [47]:
def post_processing(dc):
    dc = dc * np.logical_and(dc.MPLIA >= 27, dc.MPLIA <= 48)
    dc = dc * (dc.hbsc > (dc.wbsc + 0.5 * 2.754041))
    land_bsc_lower = dc.hbsc - 3 * dc.STD
    land_bsc_upper = dc.hbsc + 3 * dc.STD
    water_bsc_upper = dc.wbsc + 3 * 2.754041
    mask_land_outliers = np.logical_and(
        dc.sig0 > land_bsc_lower, dc.sig0 < land_bsc_upper
    )
    mask_water_outliers = dc.sig0 < water_bsc_upper
    dc = dc * (mask_land_outliers | mask_water_outliers)
    return (dc * (dc.f_post_prob > 0.8)).decision

In [48]:
flood_output = post_processing(flood_dc)

## Removal of Speckles

The following step is designed to further improve the clarity of the floodmaps. These filters do not directly relate to prior knowledge on backscattering, but consists of contextual evidence that supports, or oppose, a flood classification. This mainly targets so-called speckles. These speckles are areas of one or a few pixels, and which are likely the result of the diversity of scattering surfaces at a sub-pixel level. In this approach it is argued that small, solitary flood surfaces are unlikely. Hence speckles are removed by applying a smoothing filter which consists of a rolling window median along the x and y-axis simultaneously.


In [ ]:
flood_output = (
    flood_output.rolling({"longitude": 5, "latitude": 5}, center=True)
    .median(skipna=True)
    .persist()
)
wait(flood_output)
flood_output

## Results

In the following graphic we superimpose the data on a map and we can move the slider to see which areas become flooded over time.

In [ ]:
flood_output.hvplot.image(
    x="longitude",
    y="latitude",
    rasterize=True,
    geo=True,
    tiles=True,
    project=True,
    cmap=["rgba(0, 0, 1, 0.1)", "darkred"],
    cticks=[(0, "non-flood"), (1, "flood")],
    frame_height=400,
)

In [51]:
cluster.close(shutdown=True)